# Chaos during a stampede

Based on the paper: https://www.sciencedirect.com/science/article/pii/S1569190X20300861?casa_token=eoCj844--UoAAAAA%3AW5IEQVZQbP5ILasesMnahd3iIgfHNXfAWnaI7RsboLHrfqNA-UNxb_pyNF5FIyDN0hoKbjdC-EA 

### To do:
Implement everything

### Ideas:
Implement people as class so that we can define their state and position as parts of that class.

Use code example below, set all bordercells to 500 (= wall)
Use different values for different states of people.
Make custom update rules.

### Steps:
1. Populate board with random amount of people in random positions
2. In each time step, calculate the static floor field Sij, the dynamic floor field Dij and the event floor field Fij with help of the formulas.
3. For each pedestrian, determine his/her state and make a decision regarding movement behavior. If the pedestrian is in
state Ue,Un or Ae, calculate the transition probability according to Eqs. (15)–(17), respectively. If the pedestrian is in state An, he/
she chooses one of the strategies S1, S2 or S3 to evacuate. If the pedestrian is in state C, calculate the probability of him/her
getting up again.
4. Each pedestrian moves to his/her target cell. If the pedestrian chooses a target cell that is occupied by a fallen pedestrian, he/she will be tripped and become a new fallen pedestrian.
5. If the chaos duration is over, uninjured pedestrians regard fallen pedestrians as obstacles and conduct a normal evacuation. There will be no more new fallen pedestrians.
6. When multiple pedestrians choose to move to the same target, one pedestrian is randomly selected with equal probability to move to the target cell, and other pedestrians remain in their original cells.
7. Determine whether the evacuation is over. If there are still uninjured pedestrians in the room, return to step 2 and repeat the simulation process until all the uninjured pedestrians evacuate from the room.

### Variables:

S = static floor: describes pedestrian behavior of finding the shortest path to the exit during the evacuation process.
D = dynamic floor: represents the interactions among pedestrians in the evacuation process.
F = event floor: reflects thhat some pedestrians intend to escape from the stampede location when they are affected by the stampede.
A = risk floor field: depicts that pedestrian movement direction affects the risk of being tripped.

### evac strategies:
S1 = Tries to stay away from the stampede location (Eq 1)
S2 = Follows movement of neighbours 
S3 = keeps moving in same direction until wall is reached, then follows it clockwise or anti clockwise.
     when they stay in the same cell for more then 5 time steps, they change the desired direction randomly.


In [4]:
# imports

import numpy as np
import random

In [1]:
# functions

def init_board(r,N,e):
        # populate board with size r x r with x people
        # decide where exits are with e num of exits
        # set remaining walls to 500
    return board 

def calc_S(board):
    return S 

def calc_D(board):
    return D 

def calc_F(board):
    return F 

def calc_tumble(Rper,ka,kc,board):
    #rho = (sum of num of spaces that are occupied)/abs(U0)
    if rho > 0.64: # more then 4 people/m^2 so trample threshold is exceeded
        eps = 1
    else:
        eps = 0
     
    Aij = np.cos(theta) - 1 # theta is de hoek tussen bewegingsrichting van vorige stap en normaal van huidige cell naar cell ij
    if board[i,j] == 500:
        aij = kc*eps*rho*np.exp(ka*Aij)
    else:
        aij = 1
    
    return aij


def move_direction(person,board):

    if person_state = 'Ue':
            ....
    elif person_state = 'Un':
        ....
    elif person_state = 'Ae':
        ....
    elif person_state = 'An':
        if person.evac_strat == 'S1':
        ...
        elif person.evac_strat == 'S2':
            ... 
        else:
            ...
    elif person_state = 'C':
        # implement that they get up with probability x
        # where x follows a poisson distribution with lambda = 1 based on the duration that they are down
    else:
        raise ValueError('Non existing person state')

    return direction

def check_stampede(people_list):
    states = [person.state for person in people_list]

    if any(not(states == 'left' or states == 'C')):
        stampede = True
    else:
        stampede = False
    
    return stampede

# classes

class pedestrian:
  def __init__(self, location, state):
    self.name = name
    self.evac_strat = random.choices(['S1','S2','S3'],(50,30,20))[0]
    self.location = location
    self.state = 'Ue' # implement way to find out if they can see exit or not
    self.direction = 'up'


NameError: name 'cpl' is not defined

In [ ]:
# parameters
r = 30 # size of board
N = 100 # number of people
e = [(r/2-1):(r/2+3),0] # exit locations (middle of left wall)

Ts = 1 # occurrence time of the stampede
Tc = 50 # chaos duration

kc = 0.5 # sensitivity parameter for tumble factor 
ka = 1 # sensitivity parameter for tumble factor 
ks = 5
kd = 1
alpha = 0.3
delta = 0.3
R = 5 # perception radius for each person

Srange = 8 # stampede range

# each time step == 0.3s

In [18]:
# main function

# step 1

board = init_board(r,N,e)

stampede = True

while stampede:

    board = cellular_stampede[-1,:,:]

    # step 2
    
    S = calc_S(board)
    D = calc_D(board)
    F = calc_F(board)

    # step 3 and 4
    # if target cell is occupied state becomes C

    states = {500:'wall', 1:'..'}

    person_pos = board[...]

    next_board = board

    # step 5

    if T > Tc:
        # C becomes obstacle (= 500)
        # change evacuation strategy of others to normal
        # change to fall = 0

    # step 6
        # make list of steps with 1 current pos en 2 next pos. If multiple person have same next pos than equal chance to stay or move

    # step 7

    stampede = check_stampede(people_list)


[[[0 0 0 1 0 0 0 1 1 0]
  [1 0 0 1 0 0 1 0 0 0]
  [0 1 1 1 1 0 0 0 1 0]
  [0 0 1 1 1 1 0 0 0 0]
  [1 0 0 0 0 1 0 0 0 1]
  [1 0 1 0 1 1 1 0 0 0]
  [1 0 0 1 0 0 0 1 1 0]
  [0 1 1 1 1 1 1 1 1 1]
  [0 0 0 0 0 0 0 1 0 1]
  [0 1 1 1 0 1 0 1 1 0]]]
[[0 0 0 1 0 0 0 1 1 0]
 [1 0 0 1 0 0 1 0 0 0]
 [0 1 1 1 1 0 0 0 1 0]
 [0 0 1 1 1 1 0 0 0 0]
 [1 0 0 0 0 1 0 0 0 1]
 [1 0 1 0 1 1 1 0 0 0]
 [1 0 0 1 0 0 0 1 1 0]
 [0 1 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 1 0 1]
 [0 1 1 1 0 1 0 1 1 0]]
